## Solid Argon With LJ Potential

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [153]:
class System():
    def __init__(self):
        self.a = 3.75
        self.n = 10.
        # constants for LJ potential
        self.epsilon = 99.55 #in 1/cm
        self.sigma = 3.34 #in 1/angstrom
        
        # constants for BFW potential
        self.bfw_epsilon=98.76; self.bfw_sigma=3.76
        self.bfw_C6 = -1.10727; self.bfw_A0 = 0.27783; self.bfw_A3 = -25.2696
        self.bfw_C8 = -0.16971; self.bfw_A1 = -4.50431; self.bfw_A4 = -102.0195
        self.bfw_alpha = 12.5; self.bfw_C10 = -0.01361; self.bfw_A2 = -8.33122; self.bfw_A5 = -113.25
        self.bfw_delta = 0.01
        # constants for BBMS potential
        self.bbms_epsilon=98.76; self.bbms_sigma=3.76
        self.bbms_C6 = -1.10727; self.bbms_A0 = 0.27783; self.bbms_A3 = -25.2696
        self.bbms_C8 = -0.16971; self.bbms_A1 = -4.50431; self.bbms_A4 = -102.0195
        self.bbms_alpha = 12.5; self.bbms_C10 = -0.01361; self.bbms_A2 = -8.33122; self.bbms_A5 = -113.25
        self.bbms_delta = 0.01
        # constants for hdf potential
        self.hfd_epsilon = 99.54;self.hfd_C6 = -1.0914254;self.hfd_A = 0.9502720e7
        self.hfd_sigma =3.759; self.hfd_C8 = -0.6002595
        self.hfd_alpha = 16.345655; self.hfd_C10 = -0.3700113; self.hfd_gamma = 2.0

    def potential(self, d, vtype = "lj"):
        vval=0.
        if vtype=="lj": 
            r = d/self.sigma # to avoid recalculation of this term when calling pow()
            vval= 4*self.epsilon*(pow(r, -12) - pow(r, -6))
            
        if vtype=="bfw": 
            r = d/self.bfw_sigma
            vval= self.bfw_epsilon*(np.exp(self.bfw_alpha*(1-r))*(self.bfw_A1*(r-1)+self.bfw_A2*pow((r-1),2)+self.bfw_A3*pow((r-1),3)+self.bfw_A4*pow((r-1),4)+self.bfw_A5*pow((r-1),5))+(self.bfw_C6/(self.bfw_delta+pow(r,6))+self.bfw_C8/(self.bfw_delta+pow(r,8))+self.bfw_C10/(self.bfw_delta+pow(r,10))))
            
        if vtype=="bbms": 
            r = d/self.bbms_sigma
            vval= self.bbms_epsilon*(np.exp(self.bbms_alpha*(1-r))*(self.bbms_A1*(r-1)+self.bbms_A2*pow((r-1),2)+self.bbms_A3*pow((r-1),3)+self.bbms_A4*pow((r-1),4)+self.bbms_A5*pow((r-1),5))+(self.bbms_C6/(self.bbms_delta+pow(r,6))+self.bbms_C8/(self.bbms_delta+pow(r,8))+self.bbms_C10/(self.bbms_delta+pow(r,10)))+self.bbms_alpha*np.exp(-50.0*pow(r-1.33,2)))

        if vtype=="hfd": 
            r = d/self.hfd_sigma
            if r <= 1.4:
                vval = self.hfd_epsilon*(self.hfd_A*np.exp(-1*self.hfd_alpha*r)+(self.hfd_C6*pow(r,-6)+self.hfd_C8*pow(r,-8)+self.hfd_C10*pow(r,-10))*np.exp(-1*(pow(1.4/r,2))))
            else:
                vval = self.hfd_epsilon*(self.hfd_A*np.exp(-1*self.hfd_alpha*r)+(self.hfd_C6*pow(r,-6)+self.hfd_C8*pow(r,-8)+self.hfd_C10*pow(r,-10)))
        return vval

    def distance(self, x, y, z):
        d = (x**2 + y**2 + z**2)**0.5
        if d < self.a*1.1:
            self.closen += 1
        return d

    def lattice(self, latticeType, potentialType):
        self.closen = 0.
        r = np.linspace(-self.n, self.n, int(2.*self.n+1))
        r2 = np.linspace(-self.n/2., self.n/2., int(self.n+1))
        if latticeType=="sc":
            self.energy = self.sccell0(potentialType)
            for i, j, k in itertools.product(r,r,r):
                if (i,j,k) != (0,0,0): self.energy += self.sccell(self.a*i, self.a*j, self.a*k, potentialType)
        if latticeType == "fcc":
            self.energy = self.fcccell0(potentialType)
            for i, j, k in itertools.product(r,r,r):
                if (i, j, k) != (0, 0, 0): self.energy += self.fcccell(self.a*i, self.a*j, self.a*k, potentialType)
        if latticeType == "bcc":
            self.energy = self.bcccell0(potentialType)
            for i, j, k in itertools.product(r,r,r):
                if (i, j, k) != (0, 0, 0): self.energy += self.bcccell(self.a*i, self.a*j, self.a*k, potentialType)
        if latticeType == "hcp":
            self.energy = self.hcpcell0(potentialType)
            for i, j, k in itertools.product(r,r,r2):
                if (i,j,k) != (0,0,0): self.energy += self.hcpcell(self.a*i, self.a*j, self.a*k, potentialType)

    def sccell(self, x, y, z, potentialType):
        a=1.0
        return self.potential(self.distance(a*x, a*y, a*z), potentialType)

    def fcccell(self, x, y, z, potentialType):
        a = 2.**0.5
        return self.potential(self.distance(a*x+(y+z)/a, y/a, z/a), potentialType)

    def bcccell(self, x, y, z, potentialType):
        a = 1./(3.**0.5)
        return self.potential(self.distance(2.*a*x, 2.*a*y, 2.*a*z)) + self.potential(self.distance(2.*a*x+self.a*a, 2.*a*y+self.a*a, 2.*a*z+self.a*a), potentialType)

    def hcpcell(self, x, y, z, potentialType):
        return self.potential(self.distance(x + y/2., y/2.*(3.**0.5), 24.**0.5*z/3.), potentialType) + self.potential(self.distance(x+(y+self.a)/2., (y+self.a/3.)/2.*(3.**0.5), 24.**0.5*(z+self.a/2.)/3.), potentialType)

    def sccell0(self, potentialType):
        return 0.

    def fcccell0(self, potentialType):
        return 0.

    def bcccell0(self, potentialType):
        a = 1./2.**0.5
        return self.potential(self.distance(self.a*a, self.a*a, self.a*a), potentialType)

    def hcpcell0(self, potentialType):
        return self.potential(self.distance(self.a/2., self.a/3./2.*(3.**0.5), 6.**0.5*self.a/3.), potentialType)

In [158]:
# main body of the program
a = System()
esc_lj = []; ebcc_lj = []
efcc_lj = []; efcc_bfw = []; efcc_hfd = []; efcc_morse = []
ehcp_lj = []; ehcp_bfw = []; ehcp_hfd = []; ehcp_morse = []
x = np.linspace(3., 7., 100)
for d in x:
#     print( d )
    a.a = d
    ef = 0.
    eh = 0.
    a.lattice("sc", "lj")
    esc_lj.append(a.energy)
    a.lattice("bcc", "lj")
    ebcc_lj.append(a.energy)
    a.lattice("fcc", "lj")
    efcc_lj.append(a.energy)
    a.lattice("hcp", "lj")
    ehcp_lj.append(a.energy)

#     # a.lattice("fcc", "bfw")
#     # efcc_bfw.append(a.energy)
#     a.lattice("hcp", "bfw")
#     ehcp_bfw.append(a.energy)

#     # a.lattice("fcc", "hfd")
#     # efcc_hfd.append(a.energy)
#     a.lattice("hcp", "morse")
#     ehcp_morse.append(a.energy)

In [159]:
data=[]
trace1=go.Scatter(
            x=x,
            y=ehcp_lj,
            name='HCP'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=efcc_lj,
            name='FCC'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=esc_lj,
            name='SC'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=ebcc_lj,
            name='BCC'
            )
data.append(trace1)
layout = go.Layout(
#     title="<b>Argon L-J model</b>",
    xaxis=dict(
                title = '<b>r [1/Å]</b>',
                showline=True,
                mirror="ticks",
                ticks="inside",
                tickfont=dict(
                    size=16,
                    ),
                linewidth=2,
                tickwidth=2,
              ),        
    yaxis=dict(
                title = '<b>U(r) [1/cm]</b>',
                showline=True,
                mirror="ticks",
                ticks="inside",
                tickfont=dict(
                    size=16,
                    ),
                linewidth=2,
                tickwidth=2,
                tickprefix="  ",
#                 tickformat="1e"
    ),
    showlegend=False,
    legend=dict(
        font=dict( family='sans-serif', size=16 , color='#000' ),
        x=0.8,y=0.9
    ),
    margin=dict(
        l=80,
        r=50,
        b=65,
        t=40
    ),
)
fig2 = go.Figure(data=data, layout=layout)
py.iplot(fig2, filename='aGNR',show_link=False)

In [167]:
mnfcc,idx = min( (efcc_lj[i],i) for i in range(len(efcc_lj)) )
mnfcc,x[idx]
# min(efcc_lj)

(-1712.519233384344, 3.6464646464646466)

In [168]:
mnhcp,idx = min( (ehcp_lj[i],i) for i in range(len(ehcp_lj)) )
mnhcp,x[idx]

(-1713.2478663398813, 3.6464646464646466)

In [169]:
(mnhcp-mnfcc)/mnfcc

0.00042547431954810744

In [74]:
mn,idx = min( (esc_lj[i],i) for i in range(len(esc_lj)) )
mn,x[idx]

(-1132.4812836191609, 3.5656565656565657)

In [75]:
mn,idx = min( (ebcc_lj[i],i) for i in range(len(ebcc_lj)) )
mn,x[idx]

(-1617.4023646503922, 3.5252525252525251)

In [141]:
# main body of the program
a = System()
esc = []; ebcc = []
efcc_lj = []; efcc_bfw = []; efcc_hfd = []; efcc_morse = []
ehcp_lj = []; ehcp_bfw = []; ehcp_hfd = []; ehcp_morse = []
x = np.linspace(3., 7., 100)
for d in x:
#     print( d )
    a.a = d
    ef = 0.
    eh = 0.
    
    a.lattice("fcc", "lj")
    efcc_lj.append(a.energy)
    a.lattice("hcp", "lj")
    ehcp_lj.append(a.energy)

    a.lattice("fcc", "bfw")
    efcc_bfw.append(a.energy)
    a.lattice("hcp", "bfw")
    ehcp_bfw.append(a.energy)

    a.lattice("fcc", "hfd")
    efcc_hfd.append(a.energy)
    a.lattice("hcp", "hfd")
    ehcp_hfd.append(a.energy)


    # a.lattice("sc", "hfd")
    # esc.append(a.energy)
    # for i in range(1):
    #     #print( i )
    #     #a.a = 1.54179
    #     a.lattice("fcc", "bfw")
    #     ef += (a.energy)
    #     #a.a = 1.0902
    #     a.lattice("hcp", "bfw")
    #     eh += (a.energy)
    # efcc.append(ef/1.)
    # print( efcc[-1] )
    # a.lattice("bcc", "hfd")
    # ebcc.append(a.energy)
    # ehcp.append(eh/1.)
    # print( ehcp[-1] )


In [146]:
data=[]
trace1=go.Scatter(
            x=x,
            y=ehcp_lj,
            name='HCP-LJ'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=efcc_lj,
            name='FCC-LJ'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=ehcp_bfw,
            name='FCC-BFW'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=ehcp_bfw,
            name='HCP-BFW'
            )
data.append(trace1)

trace1=go.Scatter(
            x=x,
            y=efcc_hfd,
            name='FCC-HFD'
            )
data.append(trace1)
trace1=go.Scatter(
            x=x,
            y=ehcp_hfd,
            name='HCP-HFD'
            )
data.append(trace1)


layout = go.Layout(
#     title="<b>Argon L-J model</b>",
    xaxis=dict(
                title = '<b>r [1/Å]</b>',
                showline=True,
                mirror="ticks",
                ticks="inside",
                tickfont=dict(
                    size=16,
                    ),
                linewidth=2,
                tickwidth=2,
              ),        
    yaxis=dict(
                title = '<b>U(r) [1/cm]</b>',
                showline=True,
                mirror="ticks",
                ticks="inside",
                tickfont=dict(
                    size=16,
                    ),
                linewidth=2,
                tickwidth=2,
                tickprefix="  ",
#                 tickformat="1e"
    ),
#     showlegend=False,
    legend=dict(
        font=dict( family='sans-serif', size=16 , color='#000' ),
        x=0.8,y=0.9
    ),
    margin=dict(
        l=80,
        r=50,
        b=65,
        t=40
    ),
)
fig2 = go.Figure(data=data, layout=layout)
py.iplot(fig2, filename='aGNR',show_link=False)

In [147]:
mn1,idx = min( (efcc_bfw[i],i) for i in range(len(efcc_bfw)) )
mn1,x[idx]

(-2116.6252396133118, 3.5252525252525251)

In [148]:
mn2,idx = min( (ehcp_bfw[i],i) for i in range(len(ehcp_bfw)) )
mn2,x[idx]

(-2117.1607649426164, 3.5252525252525251)

In [151]:
(mn2-mn1)/mn1

0.00025300904443644194